<a href="https://colab.research.google.com/github/dsvalencias/MachineLearning/blob/main/Assignment_2_Bayesian_Decision_Theory_and_Parametric_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dataset from github routine

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/dsvalencias/MachineLearning/main/Datasets/diabetes.csv'
data = pd.read_csv(url)
print(data.columns.values)
data.shape

['Pregnancies' 'Glucose' 'BloodPressure' 'SkinThickness' 'Insulin' 'BMI'
 'DiabetesPedigreeFunction' 'Age' 'Outcome']


(768, 9)

**PART 1**

In [2]:
#Separating features from class label(outcome)
X_features = data.loc[:, "Pregnancies":"Age"].to_numpy()
print("X: %s %s" % (type(X_features), X_features.shape))
Y_label = data.loc[:, "Outcome"].to_numpy()
print("Y:  %s %s" % (type(Y_label), Y_label.shape))

X: <class 'numpy.ndarray'> (768, 8)
Y:  <class 'numpy.ndarray'> (768,)


In [3]:
#Separating all dataset into training and validation subsets

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, Y_label, test_size=0.33, random_state=42)

In [4]:
from sklearn import metrics
results = []

In [5]:
#import a GaussianProcessClassifier using all features -> and it should be one by one like .fit(X_train[0], y_train) .fit(X_train[1], y_train) ...
#Iterate over all features in order to clasify using just one feature at a time.
kernel = 1.0 * RBF(1.0)
for i in range(0, X_train.shape[1]):
  #Mandatory reshapes so GaussianProcessClassifier can process a single feature (column)
  single_feature_X_train = X_train[:,i].reshape(-1,1)
  single_feature_X_test = X_test[:,i].reshape(-1,1)
  #Actual model fit
  gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(single_feature_X_train, y_train)
  y_predict = gpc.predict(single_feature_X_test)
  #Get accuracy, precision and recall values for each feature
  accuracy = metrics.accuracy_score(y_test, y_predict)
  precision = metrics.precision_score(y_test, y_predict)
  recall = metrics.recall_score(y_test, y_predict)
#  accuracy_list.append(accuracy)
#  precision_list.append(precision)
#  recall_list.append(recall)
  results.append({'accuracy':accuracy, 'precision':precision, 'recall': recall})

In [6]:
#Get accuracy, precision and recall values for each feature
#print("Accuracy List\n", accuracy_list)
#print("Precision List\n", precision_list)
#print("Recall List\n", recall_list)
#print("Accuracy: %f, Precision: %f, Recall: %f" % (accuracy, precision, recall))
for i in range(0, len(results)):
  print(data.columns.values[i]+ "\n", results[i])
  print()

Pregnancies
 {'accuracy': 0.6692913385826772, 'precision': 0.5263157894736842, 'recall': 0.23255813953488372}

Glucose
 {'accuracy': 0.7204724409448819, 'precision': 0.6229508196721312, 'recall': 0.4418604651162791}

BloodPressure
 {'accuracy': 0.6614173228346457, 'precision': 0.5, 'recall': 0.05813953488372093}

SkinThickness
 {'accuracy': 0.65748031496063, 'precision': 0.4, 'recall': 0.023255813953488372}

Insulin
 {'accuracy': 0.6456692913385826, 'precision': 0.45454545454545453, 'recall': 0.23255813953488372}

BMI
 {'accuracy': 0.6811023622047244, 'precision': 0.5925925925925926, 'recall': 0.18604651162790697}

DiabetesPedigreeFunction
 {'accuracy': 0.6692913385826772, 'precision': 0.6666666666666666, 'recall': 0.046511627906976744}

Age
 {'accuracy': 0.6496062992125984, 'precision': 0.4819277108433735, 'recall': 0.46511627906976744}



In [7]:
#Draw curve for posterior distribution -> Use matplotlib for that
#Don't know what we are supposed to draw
metrics.roc_curve(y_test, y_predict)#??

(array([0.        , 0.25595238, 1.        ]),
 array([0.        , 0.46511628, 1.        ]),
 array([2, 1, 0]))

**PLAYGROUND**

In [8]:
#PlayGround

kernel = 1.0 * RBF(1.0)
#gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(X_train[:,0].reshape(-1,1), y_train)
len(results)

8